In this project, the ETL process for our company project, <b><i>Tender data Insight</b></i>, is a bit different from normal ETL process.

The data is in extremly poor quality due to the fact that the data is collected in pdf file then being converted into excel file, which will have lots of incorrect data.

This leads to lots of resources for the cleaning data process. Thus, after every finished cleaning process, the data will be loaded back into the database to prevent repeated process.

ETL Steps:
1. Import libraries
2. Explore the dataset
3. Data processing
4. Update/Load data back to SQL Server
5. Complete code (incl for loop)
**From step 1 to step 4: it will highlight the process to perform ETL process for one dataset that need to be update, however in step 5 it will include all the previous steps in order to perform ETL for numerous files that need to be loaded back to the database. 

**1. Import libraries**

In [1]:
import pandas as pd
import numpy as np

import os

from datetime import datetime, date, timedelta
import calendar

import warnings
warnings.filterwarnings('ignore')

import pypyodbc as pyodbc
from sqlalchemy import create_engine # pip install SQLAlchemy
import sqlalchemy
from sqlalchemy.engine import URL

**2. Explore the data that need to be updated**

In each file, there is two sheets equivalent to two tables need to be updated or loaded back to SQL Server<br>
- The first: the hospital tender inviting data<br>
- The second: the hospital tender winning data

In [ ]:
df_result = pd.read_excel(r'C:\JNJ_TDI_11_2023.xlsx',sheet_name='Result')
df_inviting = pd.read_excel(r'C:\JNJ_TDI_11_2023.xlsx',sheet_name='Inviting')

In [9]:
df_result.head(2)

,ID,Include/Exclude,Note,ATC3,FormulaName,Molecule,Dosage,ProductName,Visa,Manufacturer,...,AllocatedName,Province,Region,ClosedDate,ResultDate,ValidTo,TenderYear,TenderQuotationType,Status,Note_AIS
0,112023_491_R,NaN,NaN,N05A,ARIPIPRAZOLE 5MG TAB,ARIPIPRAZOLE,5MG,POZIATS 5MG,QLDB-683-18,CONG TY CP DUOC PHAM MEDISUN,...,BV AN BINH,HCMCT,HCMCT,2023-06-23,2023-10-18,2024-06-22,2023,New Tender,RAW,NaN
1,112023_610_R,NaN,NaN,N05A,OLANZAPINE 10MG TAB,OLANZAPINE,10MG,ZANOBAPINE 10,VN-16470-13,MEPRO PHARMACEUTICALS,...,BV AN BINH,HCMCT,HCMCT,2023-06-23,2023-10-18,2024-06-22,2023,New Tender,RAW,NaN


In [7]:
df_inviting.head(2)

,ID,Include/Exclude,Note,ATC3,FormulaName,Molecule,Dosage,UnitOfMeasure,Quota(SmallestUnit),TenderPackage,...,HospitalName,Province,Region,ClosedDate,ResultDate,ValidTo,TenderYear,TenderQuotationType,Status,Note_AIS
0,112023_138_I,NaN,NaN,N05A,OLANZAPINE 10MG TAB,OLANZAPINE,10MG,TABLET,300000,GENERIC 3,...,BV TAM THAN TINH HA NAM,HANAM,NORTH,2023-09-22,2023-09-22,2024-10-05,2023,Direct contracting,RAW,NaN
1,112023_139_I,NaN,NaN,N05A,QUETIAPINE 100MG TAB,QUETIAPINE,100MG,TABLET,30000,GENERIC 4,...,BV TAM THAN TINH HA NAM,HANAM,NORTH,2023-09-22,2023-09-22,2024-10-05,2023,Direct contracting,RAW,NaN


In [ ]:
#set data connection to SQL Server
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server={SERVER NAME};"
            "Database={DATABASE NAME};"
            "UID={USER ID};"
            'PWD={PASSWORD};'
        )
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': cnxn_str})
enigne = create_engine(connection_url, module=pyodbc)

**3. Data Processing**

Having said before, this data set is in extremely poor quality and needed a team to perform data cleaning. That being so, this cleaning process step involves lots of Excel operation like: <i>Fill down (Ctrl D), Sort column, vlookup, etc<i>, which is easily can mess up with the whole dataset by messing up the ID column.

For that particular reason, we have to do data processing to transform data and eliminate those causes.

In [10]:
#EDA
df_result.drop_duplicates(subset=['ID'], inplace=True)
df_result.rename(columns={'ID':'id'},inplace=True)
df_result[f'key_update_{type}'] = df_result['Quota(SmallestUnit)'].astype(str) + '_' + df_result['QuotationID'].astype(str)
df_result[f'key_update_{type}'] = df_result['Quota(SmallestUnit)'].astype(str) + '_' + df_result['QuotationID'].astype(str)
df_result['Status'] = f'Update_{datetime.now().strftime("%Y%m%d")}' #set the Status column for tracking update date
df_result['Source_Update'] = f'{datetime.now().strftime("%Y%m%d")}_' + f[:-5] #set the SourceName column for tracking the source name update
df_result.drop(['ClosedDate','ResultDate','ValidTo'], axis=1, inplace=True)

Extra all the orginal records in the dataframe from SQL database to check if the ID column is swaped or not

In [ ]:
#set data connection to SQL Server
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server={SERVER NAME};"
            "Database={DATABASE NAME};"
            "UID={USER ID};"
            'PWD={PASSWORD};'
        )
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': cnxn_str})
enigne = create_engine(connection_url, module=pyodbc)

#import data to sql to take out specific dataset that need to update
df_result.to_sql('Update_Result', con=enigne, if_exists='replace', index=False)

#export desired dataset to python
sql_Result = 'SELECT [ID],[Quota(SmallestUnit)],[QuotationID] FROM TDI_TenderResult WHERE ID IN (SELECT ID FROM Update_Result) AND [Include/Exclude] IS NULL'
sql = pd.read_sql(sql_Result, cnxn)

In order to check the status of the ID column, we create a new key column by concating the [Quota(SmallestUnit)] and the [QuotationID] in both dataframe (df_result and SQL Server). Then we will merge the new key column from SQL to the df_result. If the two values are equal we will mark that record as <b>"1"</b> and <b>"0"</b> for not equal

In [ ]:
#create key ID to verify data accuracy that is about to be updated to sql database
sql[f'key_sql_{type}'] = sql['quota(smallestunit)'].astype(str) + '_' + sql['quotationid'].astype(str)
sql[f'key_sql_{type}'] = sql[f'key_sql_{type}'].str.replace('None','0', regex=True)
df_result = pd.merge(df_result,sql[['id',f'key_sql_{type}']],on='id',how='left')
df_result = df_result[df_result[f'key_sql_{type}'].notna()]
df_result['match'] = np.where(df_result[f'key_sql_{type}'] == df_result[f'key_update_{type}'],1,0)

After that, we will check if the df_result has more than 90% of all the key matching, we can assume the rest 10% is a valid difference. Else if the df_result has less than 90% of matching key, we will eliminate all the records that are marked "0" 

In [ ]:
#if data accuracy equal or bigger than 90% then keeping the whole dataset or else delete inaccurcy records
if df_result.match.sum()/df_result.id.count() < 0.9:
    update = df_result[df_result['match']==1]

**4. Update/Load data to SQL Server**

In [ ]:
sql_update_Result = """UPDATE TDI_TenderResult
    SET TDI_TenderResult.ATC3 = Update_Result.ATC3,
            TDI_TenderResult.Molecule = Update_Result.Molecule,
            TDI_TenderResult.FormulaName = Update_Result.FormulaName,
            TDI_TenderResult.ProductName = Update_Result.ProductName,
            TDI_TenderResult.Dosage = Update_Result.Dosage,
            TDI_TenderResult.Visa = Update_Result.Visa,
            TDI_TenderResult.Manufacturer = Update_Result.Manufacturer,
            TDI_TenderResult.Nation = Update_Result.Nation,
            TDI_TenderResult.Presentation = Update_Result.Presentation,
            TDI_TenderResult.InDetail = Update_Result.InDetail,
            TDI_TenderResult.UnitOfMeasure = Update_Result.UnitOfMeasure,
            TDI_TenderResult.[Quota(SmallestUnit)] = Update_Result.[Quota(SmallestUnit)],
            TDI_TenderResult.Price = Update_Result.Price,
            TDI_TenderResult.[Value] = Update_Result.[Value],
            TDI_TenderResult.Contractor = Update_Result.Contractor,
            TDI_TenderResult.TenderPackage = Update_Result.TenderPackage,
            TDI_TenderResult.HospitalCode = Update_Result.HospitalCode,
            TDI_TenderResult.HospitalName = Update_Result.HospitalName,
            TDI_TenderResult.AllocatedCode = Update_Result.AllocatedCode,
            TDI_TenderResult.AllocatedName = Update_Result.AllocatedName,
            TDI_TenderResult.Province = Update_Result.Province,
            TDI_TenderResult.Region = Update_Result.Region,
            TDI_TenderResult.TenderYear = Update_Result.TenderYear,
            TDI_TenderResult.TenderQuotationType = Update_Result.TenderQuotationType,
            TDI_TenderResult.[Status] = Update_Result.[Status],
            TDI_TenderResult.Note_AIS =  Update_Result.Note_AIS,
            TDI_TenderResult.Source_Update = Update_Result.Source_Update 
    FROM TDI_TenderResult
    INNER JOIN Update_Result
    ON TDI_TenderResult.ID = Update_Result.ID
    DROP TABLE Update_Result"""

In [ ]:
#import the verified dataset to SQL
update.to_sql(f'Update_{type}', con=enigne, if_exists='replace', index=False)
#update the verifed dataset to SQL databse
cursor.execute(sql_update_Result)
cnxn.commit()

Note: The process is the same for the df_iniviting. Full detail view in the complete code in the <b><u>Summary</b></u> part.

**<b><u>Complete Code (incl for loop)<u>**

In [ ]:
import pandas as pd
import numpy as np

import os

from datetime import datetime, date, timedelta
import calendar

import warnings
warnings.filterwarnings('ignore')

import pypyodbc as pyodbc
from sqlalchemy import create_engine # pip install SQLAlchemy
import sqlalchemy
from sqlalchemy.engine import URL

In [2]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=VNSGNHECDWH01P;"
            "Database=Tender_Insight;"
            "UID=HEC_PBI1;"
            'PWD=doordie@2023_2;'
        )
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

In [3]:
sql_Result = 'SELECT [ID],[Quota(SmallestUnit)],[QuotationID] FROM TDI_TenderResult WHERE ID IN (SELECT ID FROM Update_Result) AND [Include/Exclude] IS NULL'

sql_Inviting = 'SELECT [ID],[Quota(SmallestUnit)],[QuotationID] FROM TDI_TenderInviting WHERE ID IN (SELECT ID FROM Update_Inviting) AND [Include/Exclude] IS NULL'

sql_update_Result = """UPDATE TDI_TenderResult
    SET TDI_TenderResult.ATC3 = Update_Result.ATC3,
            TDI_TenderResult.Molecule = Update_Result.Molecule,
            TDI_TenderResult.FormulaName = Update_Result.FormulaName,
            TDI_TenderResult.ProductName = Update_Result.ProductName,
            TDI_TenderResult.Dosage = Update_Result.Dosage,
            TDI_TenderResult.Visa = Update_Result.Visa,
            TDI_TenderResult.Manufacturer = Update_Result.Manufacturer,
            TDI_TenderResult.Nation = Update_Result.Nation,
            TDI_TenderResult.Presentation = Update_Result.Presentation,
            TDI_TenderResult.InDetail = Update_Result.InDetail,
            TDI_TenderResult.UnitOfMeasure = Update_Result.UnitOfMeasure,
            TDI_TenderResult.[Quota(SmallestUnit)] = Update_Result.[Quota(SmallestUnit)],
            TDI_TenderResult.Price = Update_Result.Price,
            TDI_TenderResult.[Value] = Update_Result.[Value],
            TDI_TenderResult.Contractor = Update_Result.Contractor,
            TDI_TenderResult.TenderPackage = Update_Result.TenderPackage,
            TDI_TenderResult.HospitalCode = Update_Result.HospitalCode,
            TDI_TenderResult.HospitalName = Update_Result.HospitalName,
            TDI_TenderResult.AllocatedCode = Update_Result.AllocatedCode,
            TDI_TenderResult.AllocatedName = Update_Result.AllocatedName,
            TDI_TenderResult.Province = Update_Result.Province,
            TDI_TenderResult.Region = Update_Result.Region,
            TDI_TenderResult.TenderYear = Update_Result.TenderYear,
            TDI_TenderResult.TenderQuotationType = Update_Result.TenderQuotationType,
            TDI_TenderResult.[Status] = Update_Result.[Status],
            TDI_TenderResult.Note_AIS =  Update_Result.Note_AIS,
            TDI_TenderResult.Source_Update = Update_Result.Source_Update 
    FROM TDI_TenderResult
    INNER JOIN Update_Result
    ON TDI_TenderResult.ID = Update_Result.ID
    DROP TABLE Update_Result"""

sql_update_Inviting = """UPDATE TDI_TenderInviting
        SET TDI_TenderInviting.ATC3 = Update_Inviting.ATC3,
            TDI_TenderInviting.Molecule = Update_Inviting.Molecule,
            TDI_TenderInviting.FormulaName = Update_Inviting.FormulaName,
            TDI_TenderInviting.Dosage = Update_Inviting.Dosage,
            TDI_TenderInviting.UnitOfMeasure = Update_Inviting.UnitOfMeasure,
            TDI_TenderInviting.[Quota(SmallestUnit)] = Update_Inviting.[Quota(SmallestUnit)],
            TDI_TenderInviting.TenderPackage = Update_Inviting.TenderPackage,
            TDI_TenderInviting.HospitalCode = Update_Inviting.HospitalCode,
            TDI_TenderInviting.HospitalName = Update_Inviting.HospitalName,
            TDI_TenderInviting.Province = Update_Inviting.Province,
            TDI_TenderInviting.Region = Update_Inviting.Region,
            TDI_TenderInviting.TenderYear = Update_Inviting.TenderYear,
            TDI_TenderInviting.TenderQuotationType = Update_Inviting.TenderQuotationType,
            TDI_TenderInviting.[Status] = Update_Inviting.[Status],
            TDI_TenderInviting.Note_AIS = Update_Inviting.Note_AIS,
            TDI_TenderInviting.Source_Update = Update_Inviting.Source_Update 
        FROM TDI_TenderInviting
        INNER JOIN Update_Inviting
        ON TDI_TenderInviting.ID = Update_Inviting.ID
        DROP TABLE Update_Inviting"""

In [4]:
path = (r'C:\\')
files = os.listdir(path)

In [5]:
types = ['Result','Inviting']
for type in types:
    connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': cnxn_str})
    enigne = create_engine(connection_url, module=pyodbc)
    for f in files:
        try:
            #read excel file for Result/Inviting sheet  
            update = pd.read_excel(path + f,sheet_name=type)
            #EDA
            update.drop_duplicates(subset=['ID'], inplace=True)
            update.rename(columns={'ID':'id'},inplace=True)
            update[f'key_update_{type}'] = update['Quota(SmallestUnit)'].astype(str) + '_' + update['QuotationID'].astype(str)
            #update = update[update['Include/Exclude'].isna()]
            update['Status'] = f'Update_{datetime.now().strftime("%Y%m%d")}'
            update['Source_Update'] = f'{datetime.now().strftime("%Y%m%d")}_' + f[:-5]
            update.drop(['ClosedDate','ResultDate','ValidTo'], axis=1, inplace=True)

            if type == 'Result':
                #EDA
                update[['Quota(SmallestUnit)','Price','Value']] = update[['Quota(SmallestUnit)','Price','Value']].astype('int',errors='ignore')
                update = update.astype('string')
                update = update.replace(['0'], np.nan)
                #import data to sql to take out specific dataset that need to update
                update.to_sql(f'Update_{type}', con=enigne, if_exists='replace', index=False)
                #export desired dataset to python
                sql = pd.read_sql(sql_Result, cnxn)
                #create key ID to verify data accuracy that is about to be updated to sql database
                sql[f'key_sql_{type}'] = sql['quota(smallestunit)'].astype(str) + '_' + sql['quotationid'].astype(str)
                sql[f'key_sql_{type}'] = sql[f'key_sql_{type}'].str.replace('None','0', regex=True)
                update = pd.merge(update,sql[['id',f'key_sql_{type}']],on='id',how='left')
                update = update[update[f'key_sql_{type}'].notna()]
                update['match'] = np.where(update[f'key_sql_{type}'] == update[f'key_update_{type}'],1,0)
                #if data accuracy equal or bigger than 90% then keeping the whole dataset or else delete inaccurcy records
                if update.match.sum()/update.id.count() < 0.9:
                    update = update[update['match']==1]
                #import the verified dataset to SQL
                update.to_sql(f'Update_{type}', con=enigne, if_exists='replace', index=False)
                #update the verifed dataset to SQL databse
                cursor.execute(sql_update_Result)
                cnxn.commit()
            
            elif type == 'Inviting':
                #EDA
                update['Quota(SmallestUnit)'] = update['Quota(SmallestUnit)'].astype('int',errors='ignore')
                update = update.astype('string')
                update = update.replace(['0'], np.nan)
                #import data to sql to take out specific dataset that need to update
                update.to_sql(f'Update_{type}', con=enigne, if_exists='replace', index=False)
                #export desired dataset to python
                sql = pd.read_sql(sql_Inviting, cnxn)
                #create key ID to verify data accuracy that is about to be updated to sql database
                sql[f'key_sql_{type}'] = sql['quota(smallestunit)'].astype(str) + '_' + sql['quotationid']
                sql[f'key_sql_{type}'] = sql[f'key_sql_{type}'].str.replace('None','0', regex=True)
                update = pd.merge(update,sql[['id',f'key_sql_{type}']],on='id',how='left')
                update = update[update[f'key_sql_{type}'].notna()]
                update['match'] = np.where(update[f'key_sql_{type}'] == update[f'key_update_{type}'],1,0)
                #if data accuracy equal or bigger than 90% then keeping the whole dataset or else delete inaccurcy records
                if update.match.sum()/update.id.count() < 0.9:
                    update = update[update['match']==1]
                #import the verified dataset to SQL
                update.to_sql(f'Update_{type}', con=enigne, if_exists='replace', index=False)
                #update the verifed dataset to SQL databse
                cursor.execute(sql_update_Inviting)
                cnxn.commit()
                
            print(f + '_' + type)

        except ValueError:
            pass
print('PLEASE DELETE ALL FILES IN THE UPDATE FILE')


Tender Data Insight_Danapha.xlsx_Result
Tender Data Insight_JnJ.xlsx_Result
Tender Data Insight_Venlafaxine.xlsx_Result
PLEASE DELETE ALL FILES IN THE UPDATE FILE
